<a href="https://colab.research.google.com/github/KAnejjar/python_projects/blob/main/Human%20Image%20Generation%20with%20Animal%20Feature/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/justinpinkney/stable-diffusion.git
%cd stable-diffusion
!pip install --upgrade pip
!pip install -r requirements.txt

!pip install --upgrade keras # on lambda stack we need to upgrade keras
!pip uninstall -y torchtext # on colab we need to remove torchtext

Cloning into 'stable-diffusion'...
remote: Enumerating objects: 1755, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 1755 (delta 3), reused 5 (delta 3), pack-reused 1747 (from 1)
Receiving objects: 100% (1755/1755), 73.93 MiB | 18.33 MiB/s, done.
Resolving deltas: 100% (1082/1082), done.
/content/stable-diffusion
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers (from -r requirements.txt (line 23))
  Cloning https://github.com/CompVis/taming-transformers.git (to revision master) to ./src/taming-transformers
  Running command git clone --filter=blob:none -

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0


In [ ]:
!nvidia-smi

Thu Dec 12 10:26:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
from PIL import Image



def preprocess_images(input_dir, output_dir, size=(256, 256)):
    os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist
    for file_name in os.listdir(input_dir):  # Iterate over files in the input directory
        img_path = os.path.join(input_dir, file_name)
        img = Image.open(img_path).convert("RGB")  # Open the image and convert to RGB
        img = img.resize(size)  # Resize the image
        img.save(os.path.join(output_dir, file_name))  # Save the preprocessed image to output_dir

# Use the correct dataset path
!ls /content/my_dataset/images/

input_path = "/content/my_dataset/images/"  # Update with your dataset path
output_path = "/content/my_dataset/preprocessed_images"  # Output directory
preprocess_images(input_path, output_path)  # Run the preprocessing function

image10.jpg  image14.jpg  image18.jpg  image21.jpg  image25.jpg  image3.jpg  image7.jpg
image11.jpg  image15.jpg  image19.jpg  image22.jpg  image26.jpg  image4.jpg  image8.jpg
image12.jpg  image16.jpg  image1.jpg   image23.jpg  image27.jpg  image5.jpg  image9.jpg
image13.jpg  image17.jpg  image20.jpg  image24.jpg  image2.jpg	 image6.jpg


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
from PIL import Image
from datasets import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
import torch

# Preprocess images: resize and save to output directory
def preprocess_images(input_dir, output_dir, size=(256, 256)):
    os.makedirs(output_dir, exist_ok=True)  # Create output directory if it doesn't exist
    for file_name in os.listdir(input_dir):  # Iterate over files in input directory
        img_path = os.path.join(input_dir, file_name)
        img = Image.open(img_path).convert("RGB")  # Open image and convert to RGB
        img = img.resize(size)  # Resize the image
        img.save(os.path.join(output_dir, file_name))  # Save preprocessed image to output_dir

# Paths for input, output, and captions
input_path = "/content/my_dataset/images/"  # Directory containing raw images
output_path = "/content/my_dataset/preprocessed_images/"  # Directory for resized images
captions_path = "/content/my_dataset/captions.txt"  # Path to captions file

# Preprocess images
preprocess_images(input_path, output_path)

# Ensure captions.txt exists
assert os.path.exists(captions_path), "The captions.txt file is missing!"

# Load captions and prepare dataset
data = []
with open(captions_path, "r") as f:
    for line in f:
        image_name, caption = line.strip().split("   ")  # Assuming '   ' separates filename and caption
        image_path = os.path.join(output_path, image_name)
        if os.path.exists(image_path):  # Ensure image exists
            data.append({"image": image_path, "text": caption})

# Convert list of dictionaries to a dictionary of lists for Hugging Face Dataset
data_dict = {
    "image": [item["image"] for item in data],
    "text": [item["text"] for item in data],
}

# Create Hugging Face Dataset
ds = Dataset.from_dict(data_dict)

# Define transformations for images
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Preprocess dataset to apply transformations
def preprocess_data(sample):
    if isinstance(sample["image"], str):  # Ensure image path is valid
        image = Image.open(sample["image"]).convert("RGB")
        sample["image"] = transform(image)  # Convert to tensor using transforms
    return sample

# Map the preprocessing function to the dataset
ds = ds.map(preprocess_data, batched=False)

# Define collate function for DataLoader
def collate_fn(batch):
    # Debugging: Print the type of each image
    for item in batch:
        if not isinstance(item["image"], torch.Tensor):
            print(f"Error: Expected tensor but got {type(item['image'])}")
            print(f"Offending item: {item['image']}")
            raise TypeError("All images must be tensors before stacking.")

    # Stack image tensors and collect captions
    images = torch.stack([item["image"] for item in batch])  # Stack image tensors
    captions = [item["text"] for item in batch]  # Collect captions
    return {"images": images, "captions": captions}


# Create DataLoader
dataloader = DataLoader(ds, batch_size=8, shuffle=True, collate_fn=collate_fn)

# Test DataLoader
for batch in dataloader:
    print("Images shape:", batch["images"].shape)  # Should be [batch_size, 3, 256, 256]
    print("Captions:", batch["captions"])
    break


Map:   0%|          | 0/27 [00:00<?, ? examples/s]

Error: Expected tensor but got <class 'list'>
Offending item: [[[0.6274510025978088, 0.6313725709915161, 0.6274510025978088, 0.6196078658103943, 0.6039215922355652, 0.5882353186607361, 0.5372549295425415, 0.48235294222831726, 0.3803921639919281, 0.2980392277240753, 0.1921568661928177, 0.09803921729326248, 0.04313725605607033, 0.027450980618596077, 0.0235294122248888, 0.0235294122248888, 0.027450980618596077, 0.0313725508749485, 0.03921568766236305, 0.07450980693101883, 0.15294118225574493, 0.2705882489681244, 0.3843137323856354, 0.45098039507865906, 0.5254902243614197, 0.5921568870544434, 0.6509804129600525, 0.6784313917160034, 0.6823529601097107, 0.6901960968971252, 0.6901960968971252, 0.6901960968971252, 0.6823529601097107, 0.6666666865348816, 0.6392157077789307, 0.6078431606292725, 0.5921568870544434, 0.5686274766921997, 0.5490196347236633, 0.5333333611488342, 0.501960813999176, 0.5098039507865906, 0.5137255191802979, 0.48627451062202454, 0.43921568989753723, 0.4000000059604645, 0.3

TypeError: All images must be tensors before stacking.